# Medicare Fraud Detection

### Data 698: Data Science Research Project - Spring 2019

### Student: Chunhui Zhu

Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2016.csv: https://data.cms.gov/Medicare-Physician-Supplier/Medicare-Provider-Utilization-and-Payment-Data-Phy/utc4-f9xp

LEIE2019.csv: https://oig.hhs.gov/exclusions/exclusions_list.asp

# PART I : Data Mining

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as plt
import pickle

### Step 1: LEIE-Fraud Data - Label Data

In [2]:
leie=pd.read_csv('LEIE2019.csv',low_memory=False)
leie.columns.values

array(['LASTNAME', 'FIRSTNAME', 'MIDNAME', 'BUSNAME', 'GENERAL',
       'SPECIALTY', 'UPIN', 'NPI', 'DOB', 'ADDRESS', 'CITY', 'STATE',
       'ZIP', 'EXCLTYPE', 'EXCLDATE', 'REINDATE', 'WAIVERDATE',
       'WVRSTATE'], dtype=object)

In [3]:
#NPI is a very important unique id to identify a provider.
npi_leie= leie[leie.NPI!= 0]

In [4]:
fraud_list=npi_leie.NPI.values.tolist()

In [5]:
len(fraud_list)

5036

There are 5,036 NPI avaliable for me to check in CMS database. 

## Step 2: CMS Data Cleaning 

Load CMS yearly dataset

trucate data set by selecting subset of columns

Standardize the columns names

Use dump data set in pickle 

### 2016 

In [6]:
def find_fraud(checklist, fraud_list):
    fraud=[]   
    for x in checklist:
        if x in fraud_list:
            fraud.append(x)
    return (fraud)

In [7]:
data2016=pd.read_table('Medicare_Provider_Util_Payment_PUF_CY2016.txt',low_memory=False)

In [8]:
print(data2016.shape)
print(data2016.columns.values)

(9714897, 26)
['NPI' 'NPPES_PROVIDER_LAST_ORG_NAME' 'NPPES_PROVIDER_FIRST_NAME'
 'NPPES_PROVIDER_MI' 'NPPES_CREDENTIALS' 'NPPES_PROVIDER_GENDER'
 'NPPES_ENTITY_CODE' 'NPPES_PROVIDER_STREET1' 'NPPES_PROVIDER_STREET2'
 'NPPES_PROVIDER_CITY' 'NPPES_PROVIDER_ZIP' 'NPPES_PROVIDER_STATE'
 'NPPES_PROVIDER_COUNTRY' 'PROVIDER_TYPE'
 'MEDICARE_PARTICIPATION_INDICATOR' 'PLACE_OF_SERVICE' 'HCPCS_CODE'
 'HCPCS_DESCRIPTION' 'HCPCS_DRUG_INDICATOR' 'LINE_SRVC_CNT'
 'BENE_UNIQUE_CNT' 'BENE_DAY_SRVC_CNT' 'AVERAGE_MEDICARE_ALLOWED_AMT'
 'AVERAGE_SUBMITTED_CHRG_AMT' 'AVERAGE_MEDICARE_PAYMENT_AMT'
 'AVERAGE_MEDICARE_STANDARD_AMT']


In [9]:
#Trucate data by selecting subset columns
data2016=data2016[['NPI', 'PROVIDER_TYPE','NPPES_PROVIDER_GENDER','HCPCS_CODE', 'LINE_SRVC_CNT', 'BENE_UNIQUE_CNT', 'BENE_DAY_SRVC_CNT','AVERAGE_MEDICARE_ALLOWED_AMT','AVERAGE_MEDICARE_PAYMENT_AMT','AVERAGE_SUBMITTED_CHRG_AMT','NPPES_PROVIDER_CITY','NPPES_PROVIDER_STATE']]
data2016.columns=['npi', 'type','gender','hcpcs_c', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt','avg_allowed_amt','avg_payment_amt','avg_submitted_chrg_amt','city','state']

In [10]:
#dump data set in pickle, 
#to avoid repeating same data processing
raw16_pkl=open("df16_pickle","wb")
pickle.dump(data2016,raw16_pkl)
raw16_pkl.close()

## 2015

In [11]:
data2015=pd.read_table('Medicare_Provider_Util_Payment_PUF_CY2015.txt',low_memory=False)

In [12]:
print(data2015.shape)
print(data2015.columns.values)

(9497892, 26)
['npi' 'nppes_provider_last_org_name' 'nppes_provider_first_name'
 'nppes_provider_mi' 'nppes_credentials' 'nppes_provider_gender'
 'nppes_entity_code' 'nppes_provider_street1' 'nppes_provider_street2'
 'nppes_provider_city' 'nppes_provider_zip' 'nppes_provider_state'
 'nppes_provider_country' 'provider_type'
 'medicare_participation_indicator' 'place_of_service' 'hcpcs_code'
 'hcpcs_description' 'hcpcs_drug_indicator' 'line_srvc_cnt'
 'bene_unique_cnt' 'bene_day_srvc_cnt' 'average_Medicare_allowed_amt'
 'average_submitted_chrg_amt' 'average_Medicare_payment_amt'
 'average_Medicare_standard_amt']


In [13]:
data2015=data2015[['npi', 'provider_type', 'nppes_provider_gender','hcpcs_code','line_srvc_cnt', 'bene_unique_cnt','bene_day_srvc_cnt', 'average_Medicare_allowed_amt', 'average_Medicare_payment_amt','average_submitted_chrg_amt','nppes_provider_city','nppes_provider_state']]
data2015.columns=['npi', 'type','gender','hcpcs_c', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt','avg_allowed_amt','avg_payment_amt','avg_submitted_chrg_amt','city','state']

In [14]:
#dump data set in pickle, 
#to avoid repeating same data processing
raw15_pkl=open("df15_pickle","wb")
pickle.dump(data2015,raw15_pkl)
raw15_pkl.close()

## 2014

In [15]:
data2014=pd.read_table('Medicare_Provider_Util_Payment_PUF_CY2014.txt',low_memory=False)

In [16]:
print(data2014.shape)
print(data2014.columns.values)

(9316308, 26)
['npi' 'nppes_provider_last_org_name' 'nppes_provider_first_name'
 'nppes_provider_mi' 'nppes_credentials' 'nppes_provider_gender'
 'nppes_entity_code' 'nppes_provider_street1' 'nppes_provider_street2'
 'nppes_provider_city' 'nppes_provider_zip' 'nppes_provider_state'
 'nppes_provider_country' 'provider_type'
 'medicare_participation_indicator' 'place_of_service' 'hcpcs_code'
 'hcpcs_description' 'hcpcs_drug_indicator' 'line_srvc_cnt'
 'bene_unique_cnt' 'bene_day_srvc_cnt' 'average_Medicare_allowed_amt'
 'average_submitted_chrg_amt' 'average_Medicare_payment_amt'
 'average_Medicare_standard_amt']


In [17]:
data2014=data2014[['npi','provider_type','nppes_provider_gender','hcpcs_code', 'line_srvc_cnt','bene_unique_cnt', 'bene_day_srvc_cnt','average_Medicare_allowed_amt','average_Medicare_payment_amt','average_submitted_chrg_amt','nppes_provider_city', 'nppes_provider_state']]
data2014.columns=['npi', 'type','gender','hcpcs_c', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt','avg_allowed_amt','avg_payment_amt','avg_submitted_chrg_amt','city','state']

In [18]:
#dump data set in pickle, 
#to avoid repeating same data processing
raw14_pkl=open("df14_pickle","wb")
pickle.dump(data2014,raw14_pkl)
raw14_pkl.close()

## 2013

In [19]:
data2013=pd.read_table('Medicare_Provider_Util_Payment_PUF_CY2013.txt',low_memory=False)

In [20]:
print(data2013.shape)
print(data2013.columns.values)

(9287877, 28)
['NPI' 'NPPES_PROVIDER_LAST_ORG_NAME' 'NPPES_PROVIDER_FIRST_NAME'
 'NPPES_PROVIDER_MI' 'NPPES_CREDENTIALS' 'NPPES_PROVIDER_GENDER'
 'NPPES_ENTITY_CODE' 'NPPES_PROVIDER_STREET1' 'NPPES_PROVIDER_STREET2'
 'NPPES_PROVIDER_CITY' 'NPPES_PROVIDER_ZIP' 'NPPES_PROVIDER_STATE'
 'NPPES_PROVIDER_COUNTRY' 'PROVIDER_TYPE'
 'MEDICARE_PARTICIPATION_INDICATOR' 'PLACE_OF_SERVICE' 'HCPCS_CODE'
 'HCPCS_DESCRIPTION' 'HCPCS_DRUG_INDICATOR' 'LINE_SRVC_CNT'
 'BENE_UNIQUE_CNT' 'BENE_DAY_SRVC_CNT' 'AVERAGE_MEDICARE_ALLOWED_AMT'
 'STDEV_MEDICARE_ALLOWED_AMT' 'AVERAGE_SUBMITTED_CHRG_AMT'
 'STDEV_SUBMITTED_CHRG_AMT' 'AVERAGE_MEDICARE_PAYMENT_AMT'
 'STDEV_MEDICARE_PAYMENT_AMT']


In [21]:
data2013=data2013[['NPI','PROVIDER_TYPE','NPPES_PROVIDER_GENDER','HCPCS_CODE', 'LINE_SRVC_CNT','BENE_UNIQUE_CNT', 'BENE_DAY_SRVC_CNT', 'AVERAGE_SUBMITTED_CHRG_AMT', 'AVERAGE_MEDICARE_PAYMENT_AMT','AVERAGE_SUBMITTED_CHRG_AMT','NPPES_PROVIDER_CITY', 'NPPES_PROVIDER_STATE']]
data2013.columns=['npi', 'type','gender','hcpcs_c', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt','avg_allowed_amt','avg_payment_amt','avg_submitted_chrg_amt','city','state']

In [22]:
#dump data set in pickle, 
#to avoid repeating same data processing
raw13_pkl=open("df13_pickle","wb")
pickle.dump(data2013,raw13_pkl)
raw13_pkl.close()

## 2012

In [23]:
data2012=pd.read_table('Medicare_Provider_Util_Payment_PUF_CY2012.txt',low_memory=False)

In [24]:
print(data2012.shape)
print(data2012.columns.values)

(9153273, 28)
['NPI' 'NPPES_PROVIDER_LAST_ORG_NAME' 'NPPES_PROVIDER_FIRST_NAME'
 'NPPES_PROVIDER_MI' 'NPPES_CREDENTIALS' 'NPPES_PROVIDER_GENDER'
 'NPPES_ENTITY_CODE' 'NPPES_PROVIDER_STREET1' 'NPPES_PROVIDER_STREET2'
 'NPPES_PROVIDER_CITY' 'NPPES_PROVIDER_ZIP' 'NPPES_PROVIDER_STATE'
 'NPPES_PROVIDER_COUNTRY' 'PROVIDER_TYPE'
 'MEDICARE_PARTICIPATION_INDICATOR' 'PLACE_OF_SERVICE' 'HCPCS_CODE'
 'HCPCS_DESCRIPTION' 'HCPCS_DRUG_INDICATOR' 'LINE_SRVC_CNT'
 'BENE_UNIQUE_CNT' 'BENE_DAY_SRVC_CNT' 'AVERAGE_MEDICARE_ALLOWED_AMT'
 'STDEV_MEDICARE_ALLOWED_AMT' 'AVERAGE_SUBMITTED_CHRG_AMT'
 'STDEV_SUBMITTED_CHRG_AMT' 'AVERAGE_MEDICARE_PAYMENT_AMT'
 'STDEV_MEDICARE_PAYMENT_AMT']


In [25]:
data2012=data2012[['NPI','PROVIDER_TYPE','NPPES_PROVIDER_GENDER','HCPCS_CODE','LINE_SRVC_CNT','BENE_UNIQUE_CNT','BENE_DAY_SRVC_CNT','AVERAGE_MEDICARE_ALLOWED_AMT','AVERAGE_MEDICARE_PAYMENT_AMT','AVERAGE_SUBMITTED_CHRG_AMT','NPPES_PROVIDER_CITY','NPPES_PROVIDER_STATE']]
data2012.columns=['npi', 'type','gender','hcpcs_c', 'line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt','avg_allowed_amt','avg_payment_amt','avg_submitted_chrg_amt','city','state']

In [26]:
#dump data set in pickle, 
#to avoid repeating same data processing
raw12_pkl=open("df12_pickle","wb")
pickle.dump(data2012,raw12_pkl)
raw12_pkl.close()

## Step 3 : Find fraud data between 2012-2016 using LEIE-Fraude Data

#### The following steps are to find the number of the fraud NPI which can be found by using LEIE mapping on CMS yearly data sets: 

-search CMS yearly data set, find the matched unique NPI from LEIE

-dump fruad NPI in pickle/store in csv files

-Total number of unique fraud NPI between 2012-2016

-Total number of unique NPI between 2012-2016

#### Reverse Method - To find new added Fraud NPI among 2012-2016

-Find 2016 fraud NPI

-Find 2015 Fraud NPI, then exclude 2016 fraud NPI 

-Find 2014 Fraud NPI, then exclude 2015 and 2016 fraud NPI

... ...


### 2016

In [27]:
df16_pkl=open("df16_pickle","rb")
data2016=pickle.load(df16_pkl)
df16_pkl.close()

In [28]:
checklist2016=data2016['npi'].unique()
len(checklist2016)

1000925

In [29]:
fraud_2016=find_fraud(checklist2016,fraud_list)
len(fraud_2016)

281

In [30]:
fraud_df16=pd.DataFrame(fraud_2016)
fraud_df16.columns=['Fraud_id_16']

fraud_df16_pkl=open("fraud_df16_pickle","wb")
pickle.dump(fraud_df16,fraud_df16_pkl)
fraud_df16_pkl.close()

### 2015

In [31]:
df15_pkl=open("df15_pickle","rb")
data2015=pickle.load(df15_pkl)
df15_pkl.close()

In [32]:
checklist2015=data2015['npi'].unique()
len(checklist2015)

968418

In [33]:
fraud_2015=find_fraud(checklist2015,fraud_list)
#print(fraud_2015)

In [34]:
len(fraud_2015)

475

In [35]:
fraud_df15=pd.DataFrame(fraud_2015)

In [36]:
fraud_df15.columns=['Fraud_id_15']

In [37]:
fraud_df15_pkl=open("fraud_df15_pickle","wb")
pickle.dump(fraud_df15,fraud_df15_pkl)
fraud_df15_pkl.close()

In [38]:
fraud_df15_pkl=open("fraud_df15_pickle","rb")
fraud_df15=pickle.load(fraud_df15_pkl)
print(fraud_df15.head(5))

   Fraud_id_15
0   1003042441
1   1003811167
2   1003817743
3   1003822834
4   1003859885


### 2014

In [39]:
df14_pkl=open("df14_pickle","rb")
data2014=pickle.load(df14_pkl)
df14_pkl.close()

In [40]:
checklist2014=data2014['npi'].unique()
len(checklist2014)

938147

In [41]:
fraud_2014=find_fraud(checklist2014,fraud_list)
#print(fraud_2014)

In [42]:
len(fraud_2014)

696

In [43]:
fraud_df14=pd.DataFrame(fraud_2014)

In [44]:
fraud_df14.columns=['Fraud_id_14']

In [45]:
fraud_df14_pkl=open("fraud_df14_pickle","wb")
pickle.dump(fraud_df14,fraud_df14_pkl)
fraud_df14_pkl.close()

In [46]:
fraud_df14_pkl=open("fraud_df14_pickle","rb")
fraud_df14=pickle.load(fraud_df14_pkl)
print(fraud_df14.head(5))

   Fraud_id_14
0   1003811167
1   1003817743
2   1003822834
3   1003859885
4   1003878711


### 2013

In [47]:
df13_pkl=open("df13_pickle","rb")
data2013=pickle.load(df13_pkl)
df13_pkl.close()

In [48]:
checklist2013=data2013['npi'].unique()
len(checklist2013)

909606

In [49]:
fraud_2013=find_fraud(checklist2013,fraud_list)
#print(fraud_2013)

[1003042441, 1003809195, 1003811167, 1003817743, 1003822834, 1003854159, 1003859885, 1003878711, 1003886979, 1003892746, 1003902800, 1003904830, 1003972670, 1003999376, 1013009729, 1013056670, 1013059740, 1013063064, 1013087741, 1013093178, 1013095975, 1013097708, 1013957182, 1013999002, 1023000122, 1023004587, 1023006129, 1023079274, 1023094190, 1023119898, 1023166410, 1023208675, 1033126651, 1033136544, 1033145487, 1033206800, 1033261953, 1033295332, 1043213887, 1043215650, 1043217052, 1043219405, 1043238421, 1043257744, 1043312168, 1043369093, 1053303792, 1053354787, 1053357376, 1053360966, 1053372201, 1053393405, 1053417345, 1053450346, 1053458018, 1053486704, 1053499673, 1063417368, 1063436517, 1063452167, 1063477503, 1063482198, 1063486090, 1063499572, 1063501823, 1063526580, 1063537215, 1063562635, 1063575561, 1063583060, 1063611499, 1063696508, 1063851285, 1073511192, 1073524823, 1073589420, 1073808945, 1083614770, 1083654826, 1083667562, 1083681423, 1083726921, 1083763874, 108

In [50]:
len(fraud_2013)

906

In [51]:
fraud_df13=pd.DataFrame(fraud_2013)

In [52]:
fraud_df13.columns=['Fraud_id_13']

In [53]:
fraud_df13_pkl=open("fraud_df13_pickle","wb")
pickle.dump(fraud_df13,fraud_df13_pkl)
fraud_df13_pkl.close()

In [54]:
#fraud_df13_pkl=open("fraud_df13_pickle","rb")
#fraud_df13=pickle.load(fraud_df13_pkl)
print(fraud_df13.head(5))

   Fraud_id_13
0   1003042441
1   1003809195
2   1003811167
3   1003817743
4   1003822834


### 2012

In [55]:
df12_pkl=open("df12_pickle","rb")
data2012=pickle.load(df12_pkl)
df12_pkl.close()

In [57]:
checklist2012=data2012['npi'].unique()
len(checklist2012)

880645

In [58]:
fraud_2012=find_fraud(checklist2012,fraud_list)
print(fraud_2012)

[1003809195, 1003811167, 1003817743, 1003822834, 1003838087, 1003854159, 1003859885, 1003878711, 1003886979, 1003902800, 1003904830, 1003908799, 1003972670, 1003999376, 1013009000, 1013009729, 1013038629, 1013056670, 1013063064, 1013087741, 1013093178, 1013095975, 1013097708, 1013297258, 1013957182, 1013998640, 1013999002, 1023000122, 1023004587, 1023006129, 1023042512, 1023079274, 1023094190, 1023119898, 1023145232, 1023166410, 1023169257, 1023208675, 1023292604, 1033126651, 1033136544, 1033145487, 1033206800, 1033206966, 1033217492, 1033230446, 1033261953, 1033274964, 1033284930, 1033295332, 1033299920, 1043215650, 1043217052, 1043219405, 1043220262, 1043238421, 1043257744, 1043302250, 1043311145, 1043312168, 1043343569, 1043369093, 1043478498, 1053303792, 1053309195, 1053334524, 1053348714, 1053354787, 1053357376, 1053360966, 1053363796, 1053372201, 1053393405, 1053417345, 1053450346, 1053458018, 1053486704, 1053499673, 1063417368, 1063436517, 1063452167, 1063482198, 1063486090, 106

In [59]:
#The number of fraud NPIs found in 2012
len(fraud_2012)

1149

In [62]:
fraud_df12=pd.DataFrame(fraud_2012)

In [63]:
fraud_df12.columns=['Fraud_id_12']

In [64]:
fraud_df12_pkl=open("fraud_df12_pickle","wb")
pickle.dump(fraud_df12,fraud_df12_pkl)
fraud_df12_pkl.close()

In [65]:
fraud_df12_pkl=open("fraud_df12_pickle","rb")
#fraud_df12=pickle.load(fraud_df12_pkl)
print(fraud_df12.head(5))

   Fraud_id_12
0   1003809195
1   1003811167
2   1003817743
3   1003822834
4   1003838087


#### Total number of pysicians between 2012-2016

In [71]:
checklist2012=pd.DataFrame(checklist2012)
checklist2013=pd.DataFrame(checklist2013)
checklist2014=pd.DataFrame(checklist2014)
checklist2015=pd.DataFrame(checklist2015)
checklist2016=pd.DataFrame(checklist2016)

In [72]:
total_id=pd.concat([checklist2012, checklist2013], axis=0, ignore_index=True)
total_id=pd.concat([total_id,checklist2014], axis=0, ignore_index=True)
total_id=pd.concat([total_id, checklist2015], axis=0, ignore_index=True)
total_id=pd.concat([total_id, checklist2016], axis=0, ignore_index=True)

In [73]:
total_id.columns=['npi']

In [74]:
total_unique=total_id['npi'].unique()

In [75]:
len(total_unique)

1208122

### Find new added Fraud NPI among 2012-2016

### 2015-2016

In [ ]:
fraud_npi_1516=[i for i, j in zip(fraud_2015,fraud_2016 ) if i != j]
len(fraud_npi_1516)

In [ ]:
#check fraud_npi_1516 list 
fraud_npi_1516

In [ ]:
#if NIP is not in fraud_2016 list, it means NPI provider has been detected by OIG in calendar year 2015
if '1003042441' not in fraud_2016:
     print("not extist")

### 2014-2015

In [ ]:
temp=[i for i, j in zip(fraud_2014,fraud_2015 ) if i != j]
fraud_npi_1415=[i for i, j in zip(temp,fraud_2016 ) if i != j]
len(fraud_npi_1415)

### 2013-2014

In [ ]:
temp=[i for i, j in zip(fraud_2013,fraud_2014) if i != j]
temp=[i for i, j in zip(temp,fraud_2015 ) if i != j]
fraud_npi_1314=[i for i, j in zip(temp,fraud_2016 ) if i != j]
len(fraud_npi_1314)

### 2012-2013

In [ ]:
temp=[i for i, j in zip(fraud_2012,fraud_2013) if i != j]
temp=[i for i, j in zip(temp,fraud_2014) if i != j]
temp=[i for i, j in zip(temp,fraud_2015 ) if i != j]i
fraud_npi_1213=[i for i, j in zip(temp,fraud_2016 ) if  != j]
len(fraud_npi_1213)

## References:

[Medicare Fee-For-Service Provider Utilization & Payment Data Physician and Other Supplier Public Use File: A Methodological Overview] last updated: May 3, 2018, The Centers of Medicare and Medicare Services, Office of Enterprise Data and Analytics https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Physician-and-Other-Supplier.html

[Medicare Provider Utilization and Payment Data Physician and Other Supplier PUF: Frequently Asked Questions] last updated: May 4, 2018, The Centers of Medicare and Medicare Services, Office of Enterprise Data and Analytics https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Physician_FAQ.pdf

[List of Excluded Individuals/Entities (LEIE)] 02-2019 Updated LEIE Database, https://oig.hhs.gov/exclusions/exclusions_list.asp; 

[The Detection of Medicare Fraud Using Machine Learning Methods with Excluded Provider Labels] by Richard A. Bauder, Taghi M. Khoshgoftaar, College of Engineering & Computer Science, Florida Atlantic University (2018)